# Satellite Acquisition

**BEFORE RUNNING THIS CODE: **Download the launch 12 GPS data [here](https://github.com/psas/Launch-12/blob/gh-pages/data/GPS/JGPS%40-32.041913222), and place it in the */resources* folder.

This notebook demonstrates the satellite acquisition proccess, in which we determine which satellites are overhead. 

## Background
This code implements the **parallel code phase search** algorithm, which is based on the mathematical concept of circular convolution. This method requires the usage of multiple fft operations, but it will find a satellite in the data (if one exists) in  single iteration. As with all of the algorithms, the parallel code phase search will need to be executed at multiple frequencies to account for the doppler shift as the satellite moves across the sky. It is possible that we will need to use a different algorithm when the code is running on an embedded platform.


## Preamble

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

##Grabs the GoldCode module from the root of the repo
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import GoldCode 
from GPSData import IQData
from Acquisition import _GetSecondLargest
from Acquisition import acquire

ModuleNotFoundError: No module named 'GoldCode'

## Import data File

As noted in the L-12 GPS data README, the sampling frequency is 4.092 MHz. and the IF is 0. Also note that the L1 signal hits the antenna at the 1.57542 GHz frequency (10.23 MHz x 154). This is dealt with by the hardware frontend, so we only have to concern ourselves with the 1.023 MHz BPSK modulated C/A code.

Additional information on file "JGPS@04.559925043" can be found at: https://github.com/psas/Launch-12/tree/gh-pages/data/GPS

Experimentation has shown that a 14ms data segment gives the best result when starting from the beginning of the launch data. The generated data only needs 1ms to get a successful acquisition. Besides speed, the issue with using a longer section of data is that a navigation data bit transition can cause the circular convolution to cancel itself out, and weaken or eliminate the acquisition of a satellite. Remember that these are 20ms apart, so if a 20ms signal section were used, the probability of a bit transition being present would be around 50 percent. Remember that a 11 or a -1-1 in the navigation message would not be a transition.

In [ ]:
# Need these to pass to importFile module
fs = 4.092*10**6 # Sampling Frequency [Hz]

NumberOfMilliseconds = 14  #14

SampleLength = NumberOfMilliseconds*10**(-3)
BytesToSkip = 0

data = IQData()

# Uncomment one of these lines to choose between Launch12 or gps-sdr-sim data
data.importFile('../resources/JGPS@04.559925043', fs, SampleLength, BytesToSkip) #L- 12 data
#data.importFile('../resources/test.max', fs, SampleLength, BytesToSkip)
#data.importFile('../resources/test4092kHz.max', fs, SampleLength, BytesToSkip)
#data.importFile('../resources/Single4092KHz5s.max', fs, SampleLength, BytesToSkip)

BinWidth = (fs/len(data.CData))
print("BinWidth is: %f [Hz]"%(BinWidth))

## Check out the imported data

We will plot the first 100 samples to see what the signal looks like.

In [ ]:
# Convert to real data
tReal, RData = data.ComplexToReal(data.CData)
plt.plot(tReal, RData)

# Set xaxis so first 100 samples are shown
xmin,xmax = plt.xlim()
plt.xlim(xmin,xmin + 100/(fs))

plt.xlabel("t (seconds)")
plt.show()

# FFT the data

We need to take the fft of the IQ datastream.

In [ ]:
# Since we are using circular-convolution, nfft must be equal to the number of samples
nfft = data.Nsamples
deltaFreq = fs/nfft
print("Calculated deltaFreq: %f [Hz]" %(deltaFreq))

# Generate frequency lists for plotting fft
# Can use f for regular fft and fshift for shifted version
fs_kHz = fs/1000 #Make easier to read
f = np.linspace (0,fs_kHz,nfft, endpoint=False)
fshift = np.linspace(-fs_kHz/2,fs_kHz/2,nfft,endpoint=False)


# Reference data size section of book to pick the len param. Should be based sample rate
ffC = np.fft.fft(data.CData, nfft)
print(nfft)

# Get a Gold Code

For example purposes, we will first search for GPS SV 15. If you review historical GPS ephemeris data available online, you can see that SV 15 is visible at the time of Launch 12.

In [ ]:
#Choose which satellite's C/A code is generated
Satellite = 15

# Generate a 1ms CA Code that repeats each chip 4 times to 
# match the sampling rate.
CACode = GoldCode.getAcquisitionCode(Satellite, 4)

# Repeat entire array for each ms of data sampled
CACodeSampled = np.tile(CACode, int(data.sampleTime*1000))

# Complex Conjugate of Gold Code fft

Take the complex conjugate of the Gold Code [real(fft) - imag(fft)j], and then multiply it by the IQ fft. If this is done correctly there should be a spike about $10^7$ times as high as the noise. 

Our main output value from this is the **pseudo-signal to noise ratio**. This is the ratio of the max peak of the result and the second highest peak. 


In [ ]:
Codefft = np.fft.fft(CACodeSampled, nfft)

GCConj = np.conjugate(Codefft)

result = np.fft.ifft(GCConj * ffC,nfft)

resultSQ = np.real(result*np.conjugate(result))

t = np.linspace(0,SampleLength,nfft,endpoint=True) 
print("Length of t: %d" %(len(t)))
print("Length of IFFT: %d" %(len(resultSQ)))

#Plotting
plt.plot(t,resultSQ)

# Zoom in to one ms
xmin,xmax = plt.xlim()
plt.xlim(xmin,xmin + 0.001)
plt.title("Cross-correlation without adjusting for doppler shift.")
plt.xlabel("Time (seconds)")
plt.show()


# Search for largest value in 1 ms worth of data
SecondLargestValue = _GetSecondLargest(resultSQ[0:int(fs*0.001)])

# Pseudo SNR
ResultsSNR = 10*np.log10(np.amax(resultSQ)/SecondLargestValue)
print("SNR: %f"%(ResultsSNR))

print("Max Value: %f, at freqshift: %d, with index %d"%(np.amax(resultSQ),0,np.argmax(resultSQ)))

## Shift frequency using multiplication by a complex exponential

Well, SV 15 seems to be nowhere to be found. The issue is that SV15 has a Doppler Shift as it moves across the sky. If it were directly overhead, we would not have to worry about it. We can go back to the historical data, and figure out that SV 15 should have a -1400Hz shift from our location. 

We can shift our data by 1400Hz by multiplying it by a complex exponential, and trying again.

$$ (I + jQ) * e^{j \omega t}$$

In [ ]:
freqShift = 1400 #-12 #[Hz] -20


# Initialize complex array
CDataShifted = data.CData*np.exp(1j*2*np.pi*freqShift*data.t)
fftCDataShifted = np.fft.fft(CDataShifted,nfft)

result = np.fft.ifft(GCConj * fftCDataShifted,nfft)
resultSQ = np.real(result*np.conjugate(result))
resultLog = 10*np.log10(resultSQ)

plt.plot(t,resultSQ)
plt.title("Cross-correlation with doppler adjustment of %d Hz"%(freqShift))
plt.xlabel("Time (seconds)")

# Zoom in to one ms
xmin,xmax = plt.xlim()
plt.xlim(xmin,xmin + 0.001)
plt.show()

# Search for largest value in 1 ms worth of data
SecondLargestValue = _GetSecondLargest(resultSQ[0:int(fs*0.001)])

# Pseudo SNR
ResultsSNR = 10*np.log10(np.amax(resultSQ)/SecondLargestValue)
print("SNR: %f"%(ResultsSNR))

print("Max Value: %f, at freqshift: %d, with index: %d"%(np.amax(resultSQ),freqShift,np.argmax(resultSQ)))

## Shift through frequency and satellite range

Now that we have seen what we should expect from the parallel code phase search, we can search across the frequency bands for doppler shifted satellites. We search 8kHz below, and 8kHz above the carrier frequency in 100Hz increments. More sensitive receivers can use larger bin sizes (500Hz or more instead of 100Hz). 

The outer loop iterates through each satellite, and executes an inner loop iterating over each frequency. The pseudo-SNR is returned from each iteration of the inner loop, and the outer loop saves the highest SNR for each satellite. 

In [ ]:
#Execute acquire function from Acquisition.py 
msForAcquisition = 14
r = acquire(data,msForAcquisition, range(-8000, 8000, 100), range(1, 33))

###### For this data sample (Launch 12, 4 seconds after launch) we should see the following satellites:

Sat | S/N (dB-Hz) | doppler shift (Hz) | code phase (chips)
:----- | -------:| -------:| --------:
 1 | 13.287713  | -1209.276720 | 394.250000
 4 | 13.700614 | 600.950356 | 547.750000
 11 | 13.381992 | 612.980296 | 514.750000
 13 | 13.554054 | -12.064141 | 655.250000
 15 | 15.276798 | -1399.910091 | 489.750000
 
The 6 strongest signals are highlighted in the output plot, and the reciever will create a channel for each one for tracking.